In [179]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder,StandardScaler, MinMaxScaler, RobustScaler
from sklearn.compose import make_column_transformer
from sklearn.ensemble import RandomForestRegressor

In [211]:
df_train = pd.read_csv('/Users/ybeen/yb/github/convince/dynamic-pricing/data_preprocessing/data/season_point.csv')

In [212]:
df_train['계절지수'] = df_train['계절지수']*10
df_train

,품목명,등급,월,평균가격,최고가격,최저가격,계절지수,카테고리,원가
0,사과 모리스_15,상,9,36463.625000,43381.250000,31325.000000,2.588596,사과,25060.000000
1,사과 모리스_15,상,10,33967.666667,40222.222222,31409.722222,2.411404,사과,25127.777778
2,사과 모리스_15,중,9,24945.220833,31095.833333,19222.916667,2.386727,사과,15378.333333
3,사과 모리스_15,중,10,27313.000000,31118.055556,22923.611111,2.613273,사과,18338.888889
4,사과 모리스_15,특,9,45311.481250,47993.750000,43443.750000,2.608181,사과,34755.000000
...,...,...,...,...,...,...,...,...,...
1960,월동배추_10.0,중,3,2400.000000,2600.000000,2200.000000,9.820972,배추,1760.000000
1961,월동배추_10.0,특,2,3212.500000,3225.000000,3200.000000,10.178218,배추,2560.000000
1962,월동배추_10.0,특,3,3100.000000,3150.000000,3050.000000,9.821782,배추,2440.000000
1963,월동배추_10.0,하,2,1700.000000,2250.000000,1150.000000,10.000000,배추,920.000000


In [196]:
# 등급, 월, 최고가격, 최저가격, 카테고리, 원가
x_train = df_train.drop(['품목명', '평균가격', '등급', '월'], axis=1)
y_train = df_train['평균가격']

In [173]:
df_train.drop(['품목명', '평균가격', '등급', '월'], axis=1).columns

Index(['최고가격', '최저가격', '계절지수', '카테고리', '원가'], dtype='object')

In [213]:
x_train = df_train.drop(['품목명', '평균가격', '등급', '월'], axis=1)
y_train = df_train['평균가격']
transformer = make_column_transformer(
    (StandardScaler(), ['최고가격', '최저가격', '원가']),
    (OneHotEncoder(), ['카테고리']),
    remainder='passthrough')
transformer.fit(x_train)
x_train = transformer.transform(x_train)

In [214]:
model = RandomForestRegressor()
model.fit(x_train, y_train)

RandomForestRegressor()

In [215]:
print(model.score(x_train, y_train))

0.9996830519512331


In [272]:
price = 6000
max_m = 0.15
min_m = 0.06
x_test = [[price + price*max_m, price + price*min_m, 3.66, '배', price]]
x_test = pd.DataFrame(x_test, columns=['최고가격', '최저가격', '계절지수', '카테고리', '원가'])
x_test = transformer.transform(x_test)

y_predict = model.predict(x_test)

In [273]:
print(y_predict[0])
print(((price + price*max_m)+(price + price*min_m))/2)
print(f'{price + price*min_m} ~ {price + price*max_m}')

7051.483848900111
6630.0
6360.0 ~ 6900.0


In [225]:
import pickle

pickle.dump(transformer, open('./transformer.pkl', 'wb'))

In [45]:
load_transformer = pickle.load(open('./minmax_scaler.pkl', 'rb'))


In [43]:
pickle.dump(model, open('new_price.pkl', 'wb'))

In [44]:
load_model = pickle.load(open('./new_price.pkl', 'rb'))

In [46]:
x_test = [['상', 8, 45000, 30000, 0.266, '사과', 25000]]
x_test = pd.DataFrame(x_test, columns=['등급', '월', '최고가격', '최저가격', '계절지수', '카테고리', '원가'])
x_test = load_transformer.transform(x_test)
y_predict = load_model.predict(x_test)

In [47]:
y_predict

array([24501.32407836])